In [ ]:
#!pip install flask

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LinearRegression

# שלב 1 – יצירת המודל (תהליך האימון)
X = np.array([50, 75, 100, 120, 150]).reshape(-1, 1)
y = np.array([500000, 750000, 1000000, 1200000, 1500000])

model = LinearRegression()
model.fit(X, y)

# שלב 2 – הגדרת אפליקציית Flask
app = Flask(__name__)

@app.route('/predict_price')
def predict_price():
    # קריאת הערך מתוך ה-URL: למשל /predict_price?size=85
    size = request.args.get('size', default=None, type=float)

    if size is None:
        return jsonify({'error': 'Missing size parameter (e.g. /predict_price?size=85)'}), 400

    prediction = model.predict([[size]])[0]
    return jsonify({'size': size, 'estimated_price': round(prediction, 2)})

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
#http://127.0.0.1:5000/predict_price?size=1

# spark

 זה לא עובד כי אין לי java

# 🧠 סיכום מסודר: Spark ו-PySpark

## 🔥 מה זה Apache Spark?
- מנוע עיבוד נתונים **מבוזר** (Distributed Processing)
- נועד לעבודה עם **Big Data** – נתונים שלא נכנסים לזיכרון
- מסוגל להריץ חישובים במקביל על אשכול מחשבים (או כמה ליבות)
- קוד פתוח – פרויקט של Apache Foundation

## 🐍 מה זה PySpark?
- הממשק של Spark בשפת **Python**
- מאפשר לכתוב קוד Python רגיל שמנצל את מנוע העיבוד של Spark

---

## 🔑 עקרונות עבודה חשובים ב-Spark:

### 1. SparkSession
- "מרכז הפיקוד" של אפליקציית Spark
- דרכו יוצרים DataFrame, טוענים קבצים ומפעילים את המנוע
```python
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()
```

### 2. Transformations (טרנספורמציות)
- פעולות שמתארות מה לעשות על הנתונים (כמו filter, groupBy)
- **לא מתבצעות בפועל** – רק נבנה גרף של פעולות
```python
df_filtered = df.filter(df["age"] > 30)  # עדיין לא בוצע
```

### 3. Actions (פעולות סופיות)
- גורמות ל-Spark להריץ את כל מה שתיארנו לפני כן
- דוגמאות: `show()`, `count()`, `collect()`
```python
df_filtered.count()  # עכשיו מתבצע בפועל
```

### 4. Lazy Evaluation (הערכה עצלה)
- Spark **לא מריץ כלום** עד שצריך תוצאה סופית (Action)
- מאפשר לו לבצע אופטימיזציה ולעבוד יעיל יותר

### 5. Partitions (מחיצות)
- Spark מחלק את הדאטה ליחידות עבודה (Partitions)
- מריץ כל חלק במקביל (אם יש כמה ליבות/מכונות)

---

## 📊 Spark לעומת Pandas:

| נושא              | Pandas                           | Spark                            |
|-------------------|----------------------------------|----------------------------------|
| טעינה לזיכרון     | כל הדאטה נטען בבת אחת          | נטען במחיצות (Partitions)       |
| גודל דאטה מתאים   | עד ~1GB (תלוי בזיכרון)          | מתאים גם לעשרות/מאות GB         |
| אופן ביצוע        | מיידי (Eager Execution)         | Lazy Evaluation (דחוי)           |
| ריצה              | ליבה אחת, מחשב מקומי            | מקבילי, תומך ב-Cluster           |
| ביצועים במחשב אישי| מהיר מאוד על קבצים קטנים        | איטי יחסית (בגלל overhead)       |

---

## 🧪 מתי להשתמש ב-Spark?
- כשיש **דאטה עצום** שלא נכנס לזיכרון
- כשעובדים בענן או ב־Cluster
- כשצריך עיבוד מקבילי, מהיר ואמין בקנה מידה גדול

---

## ⚠️ הערה חשובה:
> Spark **איטי יותר מפנדס במחשב אישי** על קבצים קטנים. הוא פורח רק כשעובדים עם Big Data אמיתי בסביבות מבוזרות.

---

## ✅ דוגמה קצרה:
```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Demo").getOrCreate()

data = [("Alice", 25), ("Bob", 17), ("Charlie", 12)]
df = spark.createDataFrame(data, ["name", "age"])

# טרנספורמציה
filtered = df.filter(df["age"] < 18)

# פעולה סופית
print(filtered.count())
```

רוצה שנוסיף גם דוגמה עם groupBy או הצצה ל-SparkML בהמשך?


# 🧠 סיכום מסודר: Spark ו-PySpark

## 🔥 מה זה Apache Spark?
- מנוע עיבוד נתונים **מבוזר** (Distributed Processing)
- נועד לעבודה עם **Big Data** – נתונים שלא נכנסים לזיכרון
- מסוגל להריץ חישובים במקביל על אשכול מחשבים (או כמה ליבות)
- קוד פתוח – פרויקט של Apache Foundation

## 🐍 מה זה PySpark?
- הממשק של Spark בשפת **Python**
- מאפשר לכתוב קוד Python רגיל שמנצל את מנוע העיבוד של Spark

---

## 🧱 מבנה Spark – שלוש שכבות עיקריות:

### 1️⃣ Spark Core
- הלב של Spark: אחראי על עיבוד מבוזר, ניהול זיכרון ומשאבים
- כולל מנועים כמו:
  - **Tungsten** – מנוע ביצועים בזיכרון
  - **Catalyst** – מנוע אופטימיזציה לשאילתות

### 2️⃣ APIs – גישות לדאטה
| API         | תיאור                                  |
|--------------|------------------------------------------|
| `RDD`        | מבנה בסיסי, גמיש אך פחות נוח           |
| `DataFrame`  | טבלה עם עמודות – כמו Pandas, הכי נפוץ ב-PySpark |
| `Dataset`    | (לא קיים בפייתון) – שילוב של DataFrame עם טיפוסי מידע ב-Java/Scala |

### 3️⃣ ספריות מתקדמות (Modules)
| ספרייה           | שימוש                                       |
|------------------|----------------------------------------------|
| **Spark SQL**     | הרצת שאילתות SQL על DataFrames             |
| **MLlib**         | ספרייה ללמידת מכונה בקנה מידה גדול         |
| **Spark Streaming** | עיבוד סטרימינג (Real Time Data)            |
| **GraphX**        | ניתוח גרפים (קיים רק ב-Scala)              |

---

## 🔑 עקרונות עבודה חשובים ב-Spark:

### 1. SparkSession
- "מרכז הפיקוד" של אפליקציית Spark
- דרכו יוצרים DataFrame, טוענים קבצים ומפעילים את המנוע
```python
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()
```

### 2. Transformations (טרנספורמציות)
- פעולות שמתארות מה לעשות על הנתונים (כמו filter, groupBy)
- **לא מתבצעות בפועל** – רק נבנה גרף של פעולות
```python
df_filtered = df.filter(df["age"] > 30)  # עדיין לא בוצע
```

### 3. Actions (פעולות סופיות)
- גורמות ל-Spark להריץ את כל מה שתיארנו לפני כן
- דוגמאות: `show()`, `count()`, `collect()`
```python
df_filtered.count()  # עכשיו מתבצע בפועל
```

### 4. Lazy Evaluation (הערכה עצלה)
- Spark **לא מריץ כלום** עד שצריך תוצאה סופית (Action)
- מאפשר לו לבצע אופטימיזציה ולעבוד יעיל יותר

### 5. Partitions (מחיצות)
- Spark מחלק את הדאטה ליחידות עבודה (Partitions)
- מריץ כל חלק במקביל (אם יש כמה ליבות/מכונות)

---

## 📊 Spark לעומת Pandas:

| נושא              | Pandas                           | Spark                            |
|-------------------|----------------------------------|----------------------------------|
| טעינה לזיכרון     | כל הדאטה נטען בבת אחת          | נטען במחיצות (Partitions)       |
| גודל דאטה מתאים   | עד ~1GB (תלוי בזיכרון)          | מתאים גם לעשרות/מאות GB         |
| אופן ביצוע        | מיידי (Eager Execution)         | Lazy Evaluation (דחוי)           |
| ריצה              | ליבה אחת, מחשב מקומי            | מקבילי, תומך ב-Cluster           |
| ביצועים במחשב אישי| מהיר מאוד על קבצים קטנים        | איטי יחסית (בגלל overhead)       |

---

## 🧪 מתי להשתמש ב-Spark?
- כשיש **דאטה עצום** שלא נכנס לזיכרון
- כשעובדים בענן או ב־Cluster
- כשצריך עיבוד מקבילי, מהיר ואמין בקנה מידה גדול

---

## ⚠️ הערה חשובה:
> Spark **איטי יותר מפנדס במחשב אישי** על קבצים קטנים. הוא פורח רק כשעובדים עם Big Data אמיתי בסביבות מבוזרות.

---

## ✅ דוגמה קצרה:
```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Demo").getOrCreate()

data = [
    ("Alice", 25),
    ("Bob", 17),
    ("Charlie", 12),
    ("Dina", 41),
    ("Eli", 17),
    ("Tamar", 33)
]

df = spark.createDataFrame(data, ["name", "age"])

# groupBy לפי גיל וספירת מופעים
grouped = df.groupBy("age").count()
grouped.show()
```

---

## ✨ הצצה ראשונית ל-SparkML:
```python
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# דאטה פשוט לדוגמה
data = [
    (1.0, 2.0),
    (2.0, 4.0),
    (3.0, 6.0),
    (4.0, 8.0)
]

df = spark.createDataFrame(data, ["x", "y"])

# הפיכת x לווקטור תכונות
vec = VectorAssembler(inputCols=["x"], outputCol="features")
df_vec = vec.transform(df)

# בניית מודל רגרסיה לינארית
lr = LinearRegression(featuresCol="features", labelCol="y")
model = lr.fit(df_vec)

# תצוגת מקדמים
print(f"מקדם השיפוע: {model.coefficients[0]}")
print(f"מקדם החיתוך: {model.intercept}")
```

רוצה שנרחיב את החלק של SparkML עם תרגיל? או דוגמה עם טסט/פרדיקציה?
